In [105]:
#This block is copied over from worksheet "03.12.18 - playing around with python"

import sqlite3
import numpy as np
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn import preprocessing

import matplotlib.pyplot as plt
import seaborn as sns; sns.set(font_scale=1.2)

# Allows charts to appear in the notebook
%matplotlib inline

import pickle


#Connection to the local database
database = "c:/Users/tommc/Documents/University/CSProject/db/baseball.db"

#This block creates a dataframe with the important (un-normalised) stats along with both IDs, with the purpose of later merging
conn = sqlite3.connect(database)

#cc is now what we use to pass commands to the database
cc = conn.cursor()

#4 things from 
cc.execute("SELECT PLAYERID, WINS_ABOVE_REPLACEMENT, EXPECTED_FIELDING_INDEPENDENT_PITCHING, EARNED_RUN_AVERAGE FROM Pitching2016 INNER JOIN PlayerIDs ON PlayerIDs.ID_FANGRAPHS = Pitching2016.PLAYERID;")

#fetchall: https://docs.python.org/2/library/sqlite3.html#sqlite3.Cursor.fetchall
testData = cc.fetchall()

#Chuck it in a numpy array in case its needed later
A1 = np.array(testData)

#making it into a df for ease of operations
df2 = pd.DataFrame(testData)
df2.columns = ["ID_RETRO", "WAR", "eFIP", "ERA"]

#Pulling the playerID table
ccc = conn.cursor()
ccc.execute("SELECT * FROM PlayerIDs;")

idData = ccc.fetchall()

#Again, making a numpy array and pf
A2 = np.array(idData)
df3 = pd.DataFrame(idData)
df3.columns = ["ID_FANGRAPHS", "ID_RETRO", "LAST_NAME", "GIVEN_NAME"]

#merging the two tables on retroID
df4 = pd.merge(df2, df3, on='ID_RETRO')
#Dropping the player names
df5 = df4.drop(['ID_RETRO', 'LAST_NAME', 'GIVEN_NAME'], axis=1)

print(np.shape(df4))



(742, 7)


In [100]:
#This is similar to the block above. It will grab the relevant information out from the Game2016 table

#d is now what we use to pass commands to the database
d = conn.cursor()

#1 represents home win, 0 away win 
d.execute("SELECT HOME_WIN, AWAY_STARTPITCH_ID, HOME_STARTPITCH_ID FROM Game2016;")

#fetchall: https://docs.python.org/2/library/sqlite3.html#sqlite3.Cursor.fetchall
pitchByGameData = d.fetchall()

#Again, making a numpy array and pf
AA1 = np.array(pitchByGameData)
df6 = pd.DataFrame(pitchByGameData)
df6.columns = ["HOME_Win", "AWAY_STARTPITCH_ID", "HOME_STARTPITCH_ID"]

#Dropping the Home starter
df7 = df6.drop(['HOME_STARTPITCH_ID'], axis=1)
df7.columns = ["HOME_Win", "ID_FANGRAPHS"]
#Same for the Away starter
df8 = df6.drop(['AWAY_STARTPITCH_ID'], axis=1)
df8.columns = ["HOME_Win", "ID_FANGRAPHS"]
print(df8)

     HOME_Win ID_FANGRAPHS
0           1     volqe001
1           0     archc001
2           1     lirif001
3           0     richg002
4           1     tillc001
5           0     hillr001
6           0     smyld001
7           1     hamec001
8           0     greiz001
9           0     tehej001
10          1     igler001
11          0     peraw001
12          0     rosst001
13          0     heana001
14          0     klubc001
15          0     younc003
16          0     tanam001
17          0     bassc001
18          1     odorj001
19          0     perem004
20          1     mills001
21          0     chenw001
22          0     nelsj004
23          1     niesj001
24          0     shiej002
25          1     gally001
26          1     carrc003
27          1     pinem001
28          1     grays001
29          1     moorm003
...       ...          ...
2398        0     skagt001
2399        0     rodre004
2400        0     shiej002
2401        0     volqe001
2402        1     sevel001
2

In [103]:
#In this block we merge what we have together before putting in a usable form

dfFINAL = pd.merge(df5, df8, on='ID_FANGRAPHS')

#Chuck it in a numpy array
unnormPitchPlayerName = np.array(dfFINAL)

unnormPitch = np.delete(unnormPitchPlayerName, 3, 1)
print(np.shape(unnormPitch))

(2428, 4)


In [104]:
#split C1 into three 1D arrays, each named for the stat.

wARstr = unnormPitch[:,0]
eFIPstr = unnormPitch[:,1]
ERAstr = unnormPitch[:,2]

wAR = wARstr.astype(np.float)
eFIP = eFIPstr.astype(np.float)
ERA = ERAstr.astype(np.float)

#Getting the wAR stat into the range [0,1] ready for MATH MAGIC
minWAR = min(wAR)
maxWAR = max(wAR)
rangeWAR = maxWAR - minWAR
#The lowest value is -1.2. This will need to change for different years
#All values are now positive
positiveWAR = wAR + 1.2
#Make the the values of wAR fall within [0,1]
wARZeroOne = positiveWAR / 7.7

#Same as the above, but for eFIP
minEFIP = min(eFIP)
maxEFIP = max(eFIP)
rangeEFIP = maxEFIP - minEFIP
#The lowest value is 2.28. This will need to change for different years
#All values are now non-negative, with smallest value 0.
positiveEFIP = eFIP - 2.28
#Make the the values of wAR fall within [0,1]
eFIPZeroOne = positiveEFIP / 6.31

#Once more, with feeling, for ERA
minERA = min(ERA)
maxERA = max(ERA)
rangeERA = maxERA - minERA
#The lowest value is 1.06. This will need to change for different years
#All values are now non-negative, with smallest value 0.
positiveERA = ERA - 1.06
#Make the the values of wAR fall within [0,1]
eRAZeroOne = positiveERA / 25.94

#Home Win column
homeWinArray = unnormPitch[:,3]

#Put this back into an array
normedPitchArray = np.column_stack((wARZeroOne, eFIPZeroOne, eRAZeroOne))

#Then back into a df
normedPitchDF = pd.DataFrame(normedPitchArray)
normedPitchDF.columns = ["WAR", "eFIP", "ERA"]
print(np.shape(normedPitchDF))

(2428, 3)


In [65]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

#Use the parameters to find out the best kernel and C value
parameters = {'kernel':('linear', 'rbf'), 'C':[0.1, 10]}
#doing a SVM!
supportVC = svm.SVC(gamma=1)
classifier = GridSearchCV(supportVC, parameters, cv=5)
#The data is held in float form in C1, the classes of each is held in intPlayoffs array
classifier.fit(normedPitchArray, homeWinArray)



GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ('linear', 'rbf'), 'C': [0.1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [87]:
#Getting the 2017 data for testing

#ccfc is now what we use to pass commands to the database
ccfc = conn.cursor()

ccfc.execute("SELECT PLAYERID, WINS_ABOVE_REPLACEMENT, EXPECTED_FIELDING_INDEPENDENT_PITCHING, EARNED_RUN_AVERAGE FROM Pitching2017 INNER JOIN PlayerIDs ON PlayerIDs.ID_FANGRAPHS = Pitching2017.PLAYERID;")

#fetchall: https://docs.python.org/2/library/sqlite3.html#sqlite3.Cursor.fetchall
gameData2017 = ccfc.fetchall()

#Chuck it in a numpy array in case its needed later
#formally A1
gameData2017Array = np.array(gameData2017)

#making it into a df for ease of operations
gameData2017Frame = pd.DataFrame(gameData2017)
gameData2017Frame.columns = ["ID_RETRO", "WAR", "eFIP", "ERA"]

#Pulling the playerID table
cccy = conn.cursor()
cccy.execute("SELECT * FROM PlayerIDs;")

playerIDData2017 = cccy.fetchall()

#Again, making a numpy array and pf
#formallyA2
playerIDData2017array = np.array(playerIDData2017)
#formally df3
playerIDData2017frame = pd.DataFrame(playerIDData2017)
playerIDData2017frame.columns = ["ID_FANGRAPHS", "ID_RETRO", "LAST_NAME", "GIVEN_NAME"]

#merging the two tables on retroID
#formally df4
mergedID2017 = pd.merge(gameData2017Frame, playerIDData2017frame, on='ID_RETRO')
#Dropping the player names
#formally df5
mergedID2017noName = mergedID2017.drop(['ID_RETRO', 'LAST_NAME', 'GIVEN_NAME'], axis=1)
print(np.shape(mergedID2017noName))

(755, 4)


In [106]:
#Get the 2017 data in to test

#dc is now what we use to pass commands to the database
dc = conn.cursor()

#1 represents home win, 0 away win 
dc.execute("SELECT HOME_WIN, AWAY_STARTPITCH_ID, HOME_STARTPITCH_ID FROM Game2017;")

#fetchall: https://docs.python.org/2/library/sqlite3.html#sqlite3.Cursor.fetchall
pitchByGameData2017 = dc.fetchall()

#Again, making a numpy array and pf
#formally AA1
pitchArray2017 = np.array(pitchByGameData2017)
#formally df6
pitchFrame2017 = pd.DataFrame(pitchByGameData2017)
pitchFrame2017.columns = ["HOME_Win", "AWAY_STARTPITCH_ID", "HOME_STARTPITCH_ID"]

#Dropping the Home starter
#formally df7
pitchFrame2017away = pitchFrame2017.drop(['HOME_STARTPITCH_ID'], axis=1)
pitchFrame2017away.columns = ["HOME_Win", "ID_FANGRAPHS"]
#Same for the Away starter
#formally df8
pitchFrame2017home = pitchFrame2017.drop(['AWAY_STARTPITCH_ID'], axis=1)
pitchFrame2017home.columns = ["HOME_Win", "ID_FANGRAPHS"]
print(np.shape(pitchFrame2017home))

(2430, 2)


In [107]:
#In this block we merge what we have together before putting in a usable form
#print(mergedID2017noName)
print(pitchFrame2017home)
dfFINAL2017 = pd.merge(mergedID2017noName, pitchFrame2017home, on='ID_FANGRAPHS')

#Chuck it in a numpy array
unnormPitchPlayerName2017 = np.array(dfFINAL2017)

unnormPitch2017 = np.delete(unnormPitchPlayerName2017, 3, 1)


     HOME_Win ID_FANGRAPHS
0           1     greiz001
1           1     martc006
2           1     archc001
3           0     felds001
4           1     kersc001
5           0     guerj003
6           1     syndn001
7           1     stras001
8           1     gausk001
9           1     porcr001
10          1     keucd001
11          1     sante001
12          1     gravk001
13          0     darvy001
14          0     corbp001
15          0     maedk001
16          0     daviz001
17          0     waina001
18          0     quinj001
19          1     mccul002
20          0     manas001
21          0     odorj001
22          0     perem004
23          1     walkt004
24          1     finnb001
25          1     hillr001
26          1     peraw001
27          0     degrj001
28          1     roart001
29          1     bundd001
...       ...          ...
2400        1     lestj001
2401        0     marqg001
2402        1     despo001
2403        0     alvah001
2404        0     cainm001
2

In [108]:
#Same as the block above but for the 2017 data
#split C1 into three 1D arrays, each named for the stat.

wARstr2017 = unnormPitch2017[:,0]
eFIPstr2017 = unnormPitch2017[:,1]
ERAstr2017 = unnormPitch2017[:,2]

wAR2017 = wARstr2017.astype(np.float)
eFIP2017 = eFIPstr2017.astype(np.float)
ERA2017 = ERAstr2017.astype(np.float)

#Getting the wAR stat into the range [0,1] ready for MATH MAGIC
minWAR2017 = min(wAR2017)
maxWAR2017 = max(wAR2017)
rangeWAR2017 = maxWAR2017 - minWAR2017
#The lowest value is -1.1. This will need to change for different years
#All values are now positive
positiveWAR2017 = wAR2017 + 1.1
#Make the the values of wAR fall within [0,1]
wARZeroOne2017 = positiveWAR2017 / 8.8

#Same as the above, but for eFIP
minEFIP2017 = min(eFIP2017)
maxEFIP2017 = max(eFIP2017)
rangeEFIP2017 = maxEFIP2017 - minEFIP2017
#The lowest value is 2.48. This will need to change for different years
#All values are now non-negative, with smallest value 0.
positiveEFIP2017 = eFIP2017 - 2.48
#Make the the values of wAR fall within [0,1]
eFIPZeroOne2017 = positiveEFIP2017 / 9.35

#Once more, with feeling, for ERA
minERA2017 = min(ERA2017)
maxERA2017 = max(ERA2017)
rangeERA2017 = maxERA2017 - minERA2017
#The lowest value is 0. This will need to change for different years
#All values are now non-negative, with smallest value 0.
positiveERA2017 = ERA2017
#Make the the values of wAR fall within [0,1]
eRAZeroOne2017 = positiveERA2017 / 27

#Home Win column
homeWinArray2017 = unnormPitch2017[:,3]

#Put this back into an array
normedPitchArray2017 = np.column_stack((wARZeroOne2017, eFIPZeroOne2017, eRAZeroOne2017))

#Then back into a df
normedPitchDF2017 = pd.DataFrame(normedPitchArray)
normedPitchDF2017.columns = ["WAR", "eFIP", "ERA"]

print(np.shape(normedPitchArray2017))

(2430, 3)


In [109]:
result1 = classifier.predict(normedPitchArray2017)

#chuck it into a numpy 
outputResults2017 = np.array(result1)
print(np.size(outputResults2017))

2430


# Did it "work"? 

Reminder:
    
The above code is for building a model based on using an array of pitching stats to see if the starting pitcher, playing at home, will win the game.
    It's absolutely longer than it needs to be, this is because I intend to use this framework to add more features and other things as the project progresses.
   
   

In [115]:
#pulling the actual results out of the db

database = "c:/Users/tommc/Documents/University/CSProject/db/baseball.db"

#This block creates a dataframe with the important (un-normalised) stats along with both IDs, with the purpose of later merging
conn = sqlite3.connect(database)

#cc is now what we use to pass commands to the database
cc = conn.cursor()

#4 things from 
cc.execute("SELECT HOME_Win FROM Game2017;")

#fetchall: https://docs.python.org/2/library/sqlite3.html#sqlite3.Cursor.fetchall
testthisData = cc.fetchall()

testthisDataArray = np.array(testthisData)

In [133]:
#Get the data in as floats
testthisDataFloat = testthisDataArray.astype(np.float)
outputResults2017Float = outputResults2017.astype(np.float)
#flatten from (2430, 1) to (2430,) to allow for comparison
testthisDataFlatten = testthisDataFloat.flatten()
outputResults2017Flatten = outputResults2017Float.flatten()

In [134]:
boolTableofComp = np.equal(testthisDataFlatten, outputResults2017Flatten)

unique, counts = np.unique(boolTableofComp, return_counts=True)
dict(zip(unique, counts))

{False: 1160, True: 1270}

# We have an answer!

In [135]:
finalScorePercent = 1270/(1160+1270)
finalScorePercent

0.522633744855967

## Pity the answer is rubbish